In [20]:
# Turning into a dataframe because easiest way I can find online to pretty format this
import pandas as pd
import numpy as np
from pickle import load
from nltk import download
from nltk.corpus import stopwords

In [21]:
with open("tokens-exploratory/exploratory_table.pickle", "rb") as handle:
    token_table = load(handle)

token_df = pd.DataFrame.from_dict(token_table, orient="index")

In [22]:
with open("tokens-exploratory/confidence_intervals.pickle", "rb") as handle:
    CI = load(handle)

In [23]:
CI = pd.Series(CI)
token_df["PPMI CI (90%)"] = CI

split_CI = pd.DataFrame(token_df["PPMI CI (90%)"].to_list(), columns=["min","max"], index=token_df.index)
token_df["PPMI Significant"] = np.where((0 < split_CI["min"]), True, False)

In [24]:
with open("embeddings-exploratory/alc_steptwo.pickle", "rb") as handle:
    embedding_table = load(handle)

embedding_df = pd.DataFrame.from_dict(embedding_table)
embedding_df = embedding_df.set_index("token")
embedding_df = embedding_df.rename(
    columns={"ground truth": "beta", "CI": "beta CI (90%)", "p-value": "beta p-value"}
)

In [28]:
df = pd.concat([token_df, embedding_df], axis=1)

# Filter out stopwords using nltk and words not used by female speakers
stop = set(stopwords.words("english"))
filtered_df = df.loc[(~df.index.isin(stop)) & (df["PPMI"] != None)]
filtered_df = filtered_df.loc[(df["M count"] > 20) & (df["F count"] > 20)]
filtered_df = filtered_df.loc[(df["beta p-value"] > 0) & (df["beta p-value"] < 0.05)]
filtered_df = filtered_df.sort_values(by=["beta p-value"])

filtered_df[0:30]

,M count,F count,total,M ratio,F ratio,F - M,PPMI,PPMI CI (90%),PPMI Significant,beta,beta CI (90%),beta p-value
proceeding,633,92,725,0.873103,0.126897,-0.746207,0.000000,"[0.0, 0.0654601790970658]",False,[0.22718515288059782],"[[0.18433788476841315], [0.21592544995400248]]",[0.001]
business,213,49,262,0.812977,0.187023,-0.625954,0.344632,"[0.0, 0.7010824056590652]",False,[0.2326399246372208],"[[0.1755752063293252], [0.21008242442227726]]",[0.001]
new,718,97,815,0.880982,0.119018,-0.761963,0.000000,"[0.0, 0.021668902851315505]",False,[0.2643007023011865],"[[0.20408719087168087], [0.24128190805037958]]",[0.001]
past,144,46,190,0.757895,0.242105,-0.515789,0.717051,"[0.27228369370689515, 1.0706526679681534]",True,[0.23934570940535055],"[[0.1911855594740814], [0.2218572876748688]]",[0.001]
point,2413,355,2768,0.871749,0.128251,-0.743497,0.000000,"[0.0, 0.0]",False,[0.21242923460607804],"[[0.1898688832248874], [0.205932871212894]]",[0.001]
name,102,48,150,0.680000,0.320000,-0.360000,1.119488,"[0.5671022403745757, 1.5067735998043177]",True,[0.21062908002045577],"[[0.1685454348060061], [0.19760183699857967]]",[0.001]
florida,135,27,162,0.833333,0.166667,-0.666667,0.178386,"[0.0, 0.7384786813074995]",False,[0.31374901379417197],"[[0.21868896818034783], [0.28109935031154176]]",[0.001]
independent,145,32,177,0.819209,0.180791,-0.638418,0.295742,"[0.0, 0.7797515058900151]",False,[0.33036405363888],"[[0.21465750717955484], [0.27580538542302596]]",[0.001]
land,478,50,528,0.905303,0.094697,-0.810606,0.000000,"[0.0, 0.0]",False,[0.23729017419847048],"[[0.1854345522203255], [0.2224659816013902]]",[0.001]
se,43,21,64,0.671875,0.328125,-0.343750,1.155667,"[0.43605896638460107, 1.628759413514183]",True,[0.4825105334658647],"[[0.336678323551034], [0.42312333130755475]]",[0.001]


In [26]:
df.loc["woman"]

M count                                                        113
F count                                                         67
total                                                          180
M ratio                                                   0.627778
F ratio                                                   0.372222
F - M                                                    -0.255556
PPMI                                                      1.337579
PPMI CI (90%)             [0.9207787708367393, 1.6723492227837264]
PPMI Significant                                              True
beta                                         [0.20967594220578925]
beta CI (90%)       [[0.18494049368511248], [0.21590977369175252]]
beta p-value                                               [0.155]
Name: woman, dtype: object

In [27]:
df.loc["man"]

M count                                                         56
F count                                                         37
total                                                           93
M ratio                                                   0.602151
F ratio                                                   0.397849
F - M                                                    -0.204301
PPMI                                                       1.43364
PPMI CI (90%)               [0.88135432254151, 1.8201207933988939]
PPMI Significant                                              True
beta                                          [0.2845088396204297]
beta CI (90%)       [[0.24424212982552576], [0.29478071903956393]]
beta p-value                                               [0.189]
Name: man, dtype: object

In [31]:
filtered_df.to_csv("concatenated_results_table.csv")